# Lista 4 de Introdução à Programação e Ciência de Dados

Professor: Rafael de Pinho

Monitor: Sillas Rocha

Aluno: Iago Dantas Figueirêdo

## Instruções Gerais
- Cada exercício deve ser implementado em um módulo separado, com funções reutilizáveis.
- Documente todas as funções com docstrings no estilo PEP 257 e use type hints.
- Utilize apenas as bibliotecas padrão do Python e o NumPy. Bibliotecas como ```pandas```, ```scipy``` e similares não são permitidas.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from threading_basics import (
    simular_traders,
    simular_feeds_de_dados,
)
from sync_locks import (
    gerenciar_risco,
    monitorar_acoes,
)

## Parte 1: Introdução ao Multithreading
Esta parte foca nos conceitos básicos de multithreading, demonstrando como criar e gerenciar threads em cenários financeiros simples. Implemente os exercícios no módulo ```threading_basics.py```.

### 1. Simulação de Traders Colocando Ordens Concorrentemente

**Função**: ```simular_traders(num_traders: int, num_ordens: int) -> Dict[str, List[Dict[str, Any]]]```

**Descrição**: Implemente uma simulação onde múltiplas threads (traders) inserem ordens de compra ou venda em uma estrutura compartilhada chamada ```order_book```, que consiste em dois dicionários: um para compras e um para vendas. Cada ordem deve conter um ID único, um preço e uma quantidade. Utilize ```threading.Lock``` para garantir que o acesso a ```order_book``` seja seguro e atômico. Após todas as threads finalizarem, retorne o estado final da estrutura ```order_book```.

**Parâmetros**:
- ```num_traders```: Número de threads (traders) a serem criadas.
- ```num_ordens```: Número de ordens que cada trader deve colocar.

**Retorno**: O estado final da estrutura ```order_book```, que é um dicionário com chaves 'buy' e 'sell', cada uma contendo uma lista de ordens (cada ordem é um dicionário com 'id', 'price' e 'quantity').

In [3]:
# Exemplo de uso da função simular_traders

num_traders = 3
num_orders = 4

order_book = simular_traders(num_traders, num_orders)

print("Livro de Ordens:")
for key in ["buy", "sell"]:
    print(f"  {key.capitalize()}:")
    for order in order_book[key]:
        print(f"    {order['id']}: {order['price']} ({order['quantity']})")

Livro de Ordens:
  Buy:
    trader_1_ordem_2: 84.39 (1000)
    trader_1_ordem_3: 96.11 (100)
    trader_2_ordem_1: 83.09 (600)
    trader_2_ordem_3: 96.87 (1000)
    trader_2_ordem_4: 96.06 (800)
  Sell:
    trader_0_ordem_1: 100.6 (500)
    trader_0_ordem_2: 104.54 (200)
    trader_0_ordem_3: 111.85 (100)
    trader_0_ordem_4: 101.96 (400)
    trader_1_ordem_1: 111.27 (900)
    trader_1_ordem_4: 113.99 (600)
    trader_2_ordem_2: 101.95 (200)


### 2. Simulação de Feeds de Dados Concorrentes

**Função**: ```simular_feeds_de_dados(acoes: List[str], tempo_total: int) -> Dict[str, float]```

**Descrição**: Crie um dicionário compartilhado ```prices``` que armazena os preços atuais de várias ações. Crie uma thread para cada ação em ```acoes```, representando um feed de dados que atualiza periodicamente (a cada $1-3$ segundos) o preço da sua ação no dicionário. Use um ```threading.Lock``` para sincronizar o acesso ao dicionário. Crie uma thread adicional que imprime os preços atuais a cada 5 segundos. A simulação deve rodar por ```tempo_total``` segundos. Retorne o dicionário final de preços após a simulação.

**Parâmetros**:
- ```acoes```: Lista de nomes de ações (e.g., ["AAPL", "GOOG", "TSLA"]).
- ```tempo_total```: Tempo total de simulação em segundos.

**Retorno**: O dicionário final de preços após a simulação.

In [4]:
# Exemplo de uso da função simular_feeds_de_dados

acoes = ["AAPL", "GOOG", "TSLA"]
tempo_total = 12  # segundos

feed_de_dados = simular_feeds_de_dados(acoes, tempo_total)

{'AAPL': 18.06, 'GOOG': 7.11, 'TSLA': 10.83}
{'AAPL': 18.75, 'GOOG': 7.24, 'TSLA': 11.22}
{'AAPL': 17.96, 'GOOG': 7.46, 'TSLA': 10.62}


## Parte 2: Sincronização com Locks

Esta parte aborda a importância da sincronização em cenários onde múltiplas threads acessam recursos compartilhados. Implemente os exercícios no módulo ```sync_locks.py```.

### 3. Gerenciamento de Risco Concorrente

**Função**: ```gerenciar_risco(total_risco: float, estrategias: List[Tuple[str, float]], tempo_total: int) -> Dict[str, float]```

**Descrição**: Suponha que há um limite total de risco ```total_risco``` para um portfólio. Cada estratégia em ```estrategias``` roda em uma thread separada e tenta alocar uma quantidade de risco. Cada thread verifica o risco total atual e, se houver espaço, aloca seu risco. Use um ```threading.Lock``` para proteger a variável de risco total. Se o risco total excederia o limite, a thread espera (usando ```time.sleep```). A simulação roda por ```tempo_total``` segundos. Retorne um dicionário com o risco alocado por cada estratégia ao final.

**Parâmetros**:
- ```total_risco```: Limite total de risco disponível.
- ```estrategias``: Lista de tuplas, onde cada tupla contém o nome da estratégia e o risco que ela deseja alocar.
- ```tempo_total```: Tempo total de simulação em segundos.

**Retorno**: Um dicionário com o risco alocado por cada estratégia.

In [5]:
# Exemplo de uso da função gerenciar_risco

total_risco = 0.30
estrategias = [
    ("e1", 0.02),
    ("e2", 0.05),
    ("e3", 0.03),
    ("e1", 0.03),
    ("e2", 0.07),
    ("e3", 0.09),
    ("e1", 0.07),
    ("e2", 0.11),
    ("e3", 0.08),
]
tempo_total = 9  # segundos

risco_alocado = gerenciar_risco(total_risco, estrategias, tempo_total)

print("\nAlocação de Risco:")
for estrategia, risco in risco_alocado.items():
    print(f"  {estrategia}: {risco:.2f}")


Alocação de Risco:
  e1: 0.05
  e2: 0.12
  e3: 0.12


### 4. Monitoramento Concorrente de Ações

**Função**: ```monitorar_acoes(acoes: List[str], valor_alvo: float) -> List[str]```

**Descrição**: Simule o monitoramento concorrente de ações utilizando múltiplas threads. Cada thread é responsável por monitorar uma ação específica da lista ```acoes```. Para simular a variação do preço, gere dois preços para cada ação: um valor anterior e um valor atual, ambos obtidos com um pequeno atraso aleatório (para simular chamadas de rede). A thread deve verificar se o ```valor_alvo``` está entre o valor anterior e o valor atual da ação (inclusive as extremidades). Se estiver, significa que o valor-alvo foi "atingido" ou ultrapassado naquele intervalo de tempo. Todas as ações que atingirem o valor-alvo devem ser adicionadas a uma lista compartilhada. Utilize ```threading.Lock``` para garantir que a lista compartilhada seja acessada de forma segura.

**Parâmetros**:
- ```acoes```: Lista de nomes de ações.
- ```valor_alvo```: Valor a ser monitorado nas oscilações de preço das ações.

**Retorno**: Lista com os nomes das ações cujo preço atingiu ou ultrapassou o valor_alvo entre o valor anterior e o valor atual.

In [6]:
# Exemplo de uso da função monitorar_acoes

acoes = ["PETR4", "VALE3", "MGLU3", "ITUB4", "ABEV3", "CASH3"]
valor_alvo = 100.0

resultado = monitorar_acoes(acoes, valor_alvo)

print("Ações:")
print(resultado)

Ação VALE3 não atingiu o valor alvo: 100.0 (Anterior: 100.78, Atual: 102.29)
Ação ITUB4 não atingiu o valor alvo: 100.0 (Anterior: 99.84, Atual: 94.92)
Ação PETR4 atingiu o valor alvo: 100.0 (Anterior: 99.05, Atual: 102.08)
Ação CASH3 não atingiu o valor alvo: 100.0 (Anterior: 100.09, Atual: 102.07)
Ação MGLU3 não atingiu o valor alvo: 100.0 (Anterior: 99.44, Atual: 97.85)
Ação ABEV3 atingiu o valor alvo: 100.0 (Anterior: 100.18, Atual: 96.73)
Ações:
['PETR4', 'ABEV3']
